In [1]:
from itertools import product
import os
import pandas as pd

In [2]:
instance_set = "bm23"
max_runtime = 3600
expected_instances = 4
degrees = [-1, 1]
perturbations = ["matrix", "objective", "rhs"]

In [35]:
# running list of strings contained by different error codes
# last two are catchalls
err = {
    "gurobipy.gurobierror: out of memory": [],
    f"could not be solved in {max_runtime} seconds. skipping.": []
}

warn = {
    "invalid value encountered in double_scalars": [],
    "could not be perturbed": [],
}

# runs that weren't run
no_go = []

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# count of instances made
count = {}

for i, file_name in enumerate(os.listdir(os.path.join("instances", instance_set))):
    
    # skip anything not a mip
    if not file_name.endswith(".mps"):
        continue
        
    # get the model name
    stem = file_name[:-4]
    
    # instantiate the count
    count[stem] = {
        (p, d): len([f for f in os.listdir(os.path.join("test_sets", instance_set, stem, f"{p}_{d}"))
                     if f.endswith(".mps")]) for p, d in product(perturbations, degrees)
    }
    
    # get the error file path
    err_pth = os.path.join("outfiles", stem + ".err")
    
    # check if the series wasn't run
    if not os.path.exists(err_pth):
        no_go.append(stem)
    
    # check if the series ran with no errors or warnings
    elif os.path.getsize(err_pth) == 0:
        empty.append(stem)
        
    else:
        # read in file
        with open(err_pth, "r") as f:
            text = f.read().lower()
            
        # check for error codes
        found_code = False
        for code in err:
            if code in text:
                err[code].append(file_name)
                found_code = True
                break
        
        if not found_code:
            for code in warn:
                if code in text:
                    warn[code].append(file_name)
                    found_code = True
                    break
                    
        if not found_code:
            other.append(file_name)

In [36]:
no_go

[]

In [37]:
len(empty) / (i + 1)

1.0

In [38]:
print(err["gurobipy.gurobierror: out of memory"])
print(len(err["gurobipy.gurobierror: out of memory"]) / (i + 1))
df = pd.read_csv("more_memory.csv", index_col=0)
for file_name in err["gurobipy.gurobierror: out of memory"]:
    current_memory = df.loc[file_name, "memory"]
    df.loc[file_name, "memory"] = min(15, current_memory * 2)
# df["memory"].astype(int)
# df.to_csv("more_memory.csv")

[]
0.0


In [39]:
len(err[f"could not be solved in {max_runtime} seconds. skipping."]) / (i + 1)

0.0

In [40]:
len(warn["invalid value encountered in double_scalars"]) / (i + 1)

0.0

In [41]:
len(warn["could not be perturbed"]) / (i + 1)

0.0

In [42]:
print(other)
len(other) /( i + 1)

[]


0.0

In [44]:
{stem: {(p, d): count for (p, d), count in count[stem].items() if count < expected_instances} for stem in count}

{'bm23': {}}